# Домашнее задание № 7

### Задание 1 Реализовать алгоритм Леска и проверить его на реальном датасете (8 баллов)

Ворднет можно использовать для дизамбигуации. Самый простой алгоритм дизамбигуации - алгоритм Леска. В нём нужное значение слова находится через пересечение слов контекста, в котором употреблено это слово, с определениями значений слова из ворднета. Значение с максимальным пересечением - нужное.

Реализуйте его

In [3]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


True

In [ ]:
def normalize(text):
    
    words = [token.text.strip(punct) for token in list(razdel_tokenize(text))]
    words = [morph.parse(word)[0].normal_form for word in words if word and word not in stops]

    return words

In [6]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\omw-1.4.zip.


True

In [4]:
from nltk.corpus import wordnet as wn

In [9]:
from nltk.tokenize import word_tokenize

In [15]:
from nltk.corpus import stopwords

In [11]:
word_tokenize('hi there! msie o"braian',)

['hi', 'there', '!', 'msie', 'o', "''", 'braian']

In [17]:
stops = set(stopwords.words('english'))

In [121]:
punct = punctuation+'«»—…“”*№–.!?-'

In [41]:
def normalize(text):
    words = [token.strip(punct) for token in word_tokenize(text)]
    words = [lemmatizer.lemmatize(word) for word in words if word and word not in stops]

    return words

In [12]:
from nltk.stem import WordNetLemmatizer
 
lemmatizer = WordNetLemmatizer()


rocks : rock
corpora : corpus


In [67]:
def get_words_in_context_wiki(sentence):
    word_context = []
    for i in range(len(sentence)):
        if sentence[i][0]:
            word_context.append((sentence[i][1], sentence[i][0], ' '.join([word[1] for word in sentence if word[1]!=sentence[i][1] and word[1] not in punct])))
    return word_context

In [76]:
def lesk( word, sentence ):
    synsets = wn.synsets(word)

    bestsense = 0
    maxoverlap = 0
    sentence = set(sentence.split())
    
    for i, syns in enumerate(synsets):
        descr = set(normalize(syns.definition()))
        if len(descr & sentence) > maxoverlap:
            bestsense = i
            maxoverlap = len(descr & sentence)

        
        
    return bestsense


In [64]:
def get_words_in_context(word, sentence): #для удобства немного подредактировала функцию и принцип вывода
    sentence = normalize(sentence)
    sentence = [i for i in sentence if i!=word]
    return word, ' '.join(sentence)

In [20]:
corpus_wsd = []
corpus = open('corpus_wsd_50k.txt').read().split('\n\n')
for sent in corpus:
    corpus_wsd.append([s.split('\t') for s in sent.split('\n')])

In [87]:


    total = 0
    correct = 0
    for numb, i in enumerate(corpus_wsd):
        if numb % 10000==0:
            print(numb)
        ambiwords = get_words_in_context_wiki(i)
        for q in ambiwords:
            total+=1
            word, gold, context = q
            predicted = lesk(word, context)
            correct += int(wn.lemma_from_key(gold).synset()==wn.synsets(word)[predicted])


0
10000
20000
30000
40000


Вполне хороший результат для настолько наивного алгоритма! 

In [89]:
correct/total

0.4306519446632738

In [92]:
import adagram

### Задание 2* (2 балла)

В семинаре для WSI на данных Диалога использовался только Fastext. Попробуйте заменить его на адаграм (обучите свою модель или используйте предобученную out.pkl или https://s3.amazonaws.com/kostia.lopuhin/all.a010.p10.d300.w5.m100.nonorm.slim.joblib), а также поэкспериментируйте с разными алгоритмами кластеризации и их параметрами (можно использовать только те алгоритмы, которые обучаются достаточно быстро)

Для каждого эксперимента рассчитайте ARI

In [102]:
#!wget 'https://s3.amazonaws.com/kostia.lopuhin/all.a010.p10.d300.w5.m100.nonorm.slim.joblib'
vm = adagram.VectorModel.load('all.a010.p10.d300.w5.m100.nonorm.slim.joblib')

In [134]:
len(vm.sense_vector('мир', vm.disambiguate("мир", ['существование', "космический"]).argmax()))

300

In [109]:
vm.sense_neighbors('бор',0)

c:\Users\user\OneDrive\Рабочий стол\учеба\notebooks\hse\lib\site-packages\adagram\model.py:124: RuntimeWarning: invalid value encountered in true_divide
  sim_matrix = np.dot(self.In, s_v) / self.InNorms


[('заречье', 0, 0.6043546),
 ('сосновка', 0, 0.5947289),
 ('борок', 0, 0.5784992),
 ('волосово', 0, 0.572816),
 ('деревня', 5, 0.56453264),
 ('александровка', 0, 0.55563515),
 ('андреевка', 0, 0.5535462),
 ('поселок', 2, 0.5489396),
 ('роща', 0, 0.5486376),
 ('михайловка', 0, 0.5477818)]

In [99]:
import pandas as pd

In [100]:
df = pd.read_csv('train.csv', sep='\t')

In [105]:
from pymorphy2 import MorphAnalyzer

In [106]:
morph = MorphAnalyzer()

rus_stops = set(stopwords.words('russian'))

def normalize_rus(text):
    
    words = [token.strip(punct) for token in text.split()]
    words = [morph.parse(word)[0].normal_form for word in words if word and word not in rus_stops]

    return words

In [122]:
df['normalized_context']=df['context'].apply(normalize_rus)

In [123]:
grouped_df = df.groupby('word')[['word', 'context', 'gold_sense_id', 'normalized_context']]

In [119]:
import numpy as np

In [120]:
from string import punctuation


In [124]:
from sklearn.metrics import adjusted_rand_score

In [129]:
import warnings
warnings.filterwarnings('ignore')

In [132]:
from sklearn.cluster import *

In [143]:
def get_embedding_adagram(text, model, window, dim):
    
    
    word2context = []
    for i in range(len(text)-1):
        left = max(0, i-window)
        word = text[i]
        left_context = text[left:i]
        right_context = text[i+1:i+window]
        context = left_context + right_context
        word2context.append((word, context))
    
    
    
    vectors = np.zeros((len(word2context), dim))
    
    for i,word in enumerate(word2context):
        word, context = word
        try:
            sense = model.disambiguate(word, context).argmax()
            v = model.sense_vector(word, sense)
            vectors[i] = v 

        except (KeyError):
            continue
    
    if vectors.any():
        vector = np.average(vectors, axis=0)
    else:
        vector = np.zeros((dim))
    
    return vector
        

Как я поняла, требуется векторизовать именно контекст слова,  а не напрямую дизамбигуировать его с помощью адаграмма (к слову, попытки сделать это принесли отрицательный ARI)

Чтобы не извлекать вектора каждый раз, было решено провести все эксперименты в одном цикле.

In [152]:
ARI = dict()

for key, _ in grouped_df:

    texts = grouped_df.get_group(key)['context']
    X = np.zeros((len(texts), 300))
    
    for i, text in enumerate(texts):
        text = [word for word in text if word != key]
        X[i] = get_embedding_adagram(text, vm, 10, 300)
        
    cluster_af_strict = AffinityPropagation(damping=0.9)
    cluster_af_chill = AffinityPropagation(damping=0.6)
    cluster_kmeans_2 = KMeans(2)
    cluster_kmeans_3 = KMeans(3)
    cluster_dbscan_more_clusters = DBSCAN(min_samples=3, eps=0.1)
    cluster_dbscan_less_clusters = DBSCAN(min_samples=15, eps=0.3)
    cluster_models = [cluster_af_strict, cluster_af_chill, cluster_kmeans_2, cluster_kmeans_3, cluster_dbscan_more_clusters, cluster_dbscan_less_clusters,]
    cl_dict = dict().fromkeys([str(cl) for cl in cluster_models])
    ARI[key]=cl_dict
    for cl in cluster_models:
        cl.fit(X)
        labels = np.array(cl.labels_)+1
        cl_dict[str(cl)]=adjusted_rand_score(grouped_df.get_group(key)['gold_sense_id'], labels)
    
#print(np.mean(ARI))

In [153]:
ARI

{'бор': {'AffinityPropagation(damping=0.9)': 0.060499816543682985,
  'AffinityPropagation(damping=0.6)': 0.060499816543682985,
  'KMeans(n_clusters=2)': 0.021966527196652718,
  'KMeans(n_clusters=3)': 0.05879342190177359,
  'DBSCAN(eps=0.1, min_samples=3)': -0.013878836902379838,
  'DBSCAN(eps=0.3, min_samples=15)': 0.0},
 'замок': {'AffinityPropagation(damping=0.9)': 0.0,
  'AffinityPropagation(damping=0.6)': 0.020073820116747968,
  'KMeans(n_clusters=2)': 0.10852801674075456,
  'KMeans(n_clusters=3)': 0.1279994083623275,
  'DBSCAN(eps=0.1, min_samples=3)': 0.12965894306862152,
  'DBSCAN(eps=0.3, min_samples=15)': 0.023480134669933688},
 'лук': {'AffinityPropagation(damping=0.9)': 0.026059548909006133,
  'AffinityPropagation(damping=0.6)': 0.017322267711951715,
  'KMeans(n_clusters=2)': 0.05819292170507473,
  'KMeans(n_clusters=3)': 0.06946293941409047,
  'DBSCAN(eps=0.1, min_samples=3)': 0.09074873568109885,
  'DBSCAN(eps=0.3, min_samples=15)': 0.05819292170507473},
 'суда': {'Affini

Способ складирования ARI в зависимости от алгоритма кластеризации мною был выбран не самый удачный, так что с помощью еще одного цикла настроим удобный принт. И наши финалисты:

1) K-means(3)
2) K-means(2)
3) DBSCAN(eps=0.1, minsamples=3)

В целом удалось улучшить результат аж в 8 раз, но показатель все еще мизерный.

In [155]:
for i in cl_dict.keys():
    aris = []
    for key, value in ARI.items():
        aris.append(value[i])
    print(i+': '+str(np.mean(aris)))

AffinityPropagation(damping=0.9): 0.02558523720473241
AffinityPropagation(damping=0.6): 0.028419371934655793
KMeans(n_clusters=2): 0.07250770114421909
KMeans(n_clusters=3): 0.08516351118800852
DBSCAN(eps=0.1, min_samples=3): 0.08419388386627949
DBSCAN(eps=0.3, min_samples=15): 0.013699585126716946
